In [ ]:
import os
import requests

def submit_to_passer(pdb_path, model):
    url = "https://passer.smu.edu/api"
    files = {'pdbFile': open(pdb_path, 'rb')}
    data = {"model": model, "format": "json"}

    try:
        response = requests.post(url, files=files, data=data)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        return {"error": str(e)}

def run_batch_passer(pdb_dir):
    models = ["ensemble", "automl", "rank"]
    result_file = os.path.join(pdb_dir, "passer_results.txt")

    # Create (or overwrite) the output file
    with open(result_file, "w") as out:
        out.write("PASSer batch submission results\n")
        out.write("=" * 40 + "\n\n")

    # Start appending results
    for filename in os.listdir(pdb_dir):
        if filename.endswith("_auto.pdb"):
            pdb_path = os.path.join(pdb_dir, filename)
            with open(result_file, "a") as out:
                out.write(f"\n=== Results for {filename} ===\n")
                for model in models:
                    out.write(f"\n-- Model: {model} --\n")
                    result = submit_to_passer(pdb_path, model)
                    out.write(str(result) + "\n")

    print(f"PASSer results saved to {result_file}")

# Example usage
run_batch_passer("/home/cloetes-lab/amia/test/variant_outputs")  # Replace with your actual directory



📁 Submitting: /home/cloetes-lab/amia/test/variant_outputs/K215N_auto.pdb
📁 Submitting: /home/cloetes-lab/amia/test/variant_outputs/I84M_auto.pdb
📁 Submitting: /home/cloetes-lab/amia/test/variant_outputs/T210A_auto.pdb


KeyboardInterrupt: 

In [4]:
import re
import json
import ast
import os

def parse_passer_batch_file(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"[ERROR] File not found: {filepath}")

    results = []
    current_pdb = None
    current_model = None

    with open(filepath, "r") as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Skip header or empty/separator lines
        if not line or line.startswith("PASSer") or set(line) == {"="}:
            i += 1
            continue

        # Detect new PDB result block
        if line.startswith("=== Results for"):
            current_pdb = line.replace("=== Results for", "").replace("===", "").strip()

        # Detect model section
        elif line.startswith("-- Model:"):
            current_model = line.replace("-- Model:", "").replace("--", "").strip()

            # Read next line as model dictionary
            i += 1
            if i < len(lines):
                dict_line = lines[i].strip()

                # Remove surrounding quotes if present
                if dict_line.startswith('"') and dict_line.endswith('"'):
                    dict_line = dict_line[1:-1]

                try:
                    parsed_obj = ast.literal_eval(dict_line)

                    if not isinstance(parsed_obj, dict):
                        raise ValueError("Parsed object is not a dictionary")

                    for pocket_id, pocket_data in parsed_obj.items():
                        prob = pocket_data.get("prob", "")
                        residue_expr = pocket_data.get("residues", "")

                        # Extract chain and residue IDs
                        residue_dict = {}
                        chains = set()
                        matches = re.findall(r"\(chain (\w) and resid ([^)]+)\)", residue_expr)

                        for chain, res_str in matches:
                            residue_ids = []
                            for r in res_str.strip().split():
                                try:
                                    residue_ids.append(int(r))
                                except ValueError:
                                    continue
                            chains.add(chain)
                            residue_dict[chain] = residue_ids

                        results.append({
                            "pdb": current_pdb,
                            "model": current_model,
                            "pocket": pocket_id,
                            "probability": prob,
                            "chains": sorted(list(chains)),
                            "residues": residue_dict
                        })

                except Exception as e:
                    print(f"[ERROR] Failed to parse dict for {current_pdb}, model {current_model}: {e}")
                    print(f"[DEBUG] Line content: {dict_line}")

        i += 1

    return results


# === RUN AND SAVE PARSED OUTPUT ===

if __name__ == "__main__":
    file_path = "/home/cloetes-lab/amia/test/variant_outputs/passer_results.txt"  # Update if needed
    parsed = parse_passer_batch_file(file_path)

    if parsed:
        # Determine output path: same directory as input file + "json_auto.txt"
        output_dir = os.path.dirname(file_path)
        output_file = os.path.join(output_dir, "json_auto.txt")

        with open(output_file, "w") as out_f:
            json.dump(parsed, out_f, indent=2)

        print(f"Parsed results saved to {output_file}")
    else:
        print("No valid results were parsed.")



Parsed results saved to /home/cloetes-lab/amia/test/variant_outputs/json_auto.txt


In [5]:
import json
import pandas as pd
from collections import defaultdict
from pathlib import Path

# === Load JSON data ===
with open("/home/cloetes-lab/amia/test/variant_outputs/json_auto.txt", encoding="utf-8") as f:
    data = json.load(f)

# === Organize into nested dict: pdb → pocket → model → cell content ===
pdb_tables = defaultdict(lambda: defaultdict(dict))  # {pdb: {pocket: {model: cell_content}}}

for entry in data:
    pdb = entry["pdb"]
    model = entry["model"]
    pocket = f'Pocket {entry["pocket"]}'

    # Parse probability
    try:
        prob = float(entry["probability"].replace('%', ''))
        prob_str = f"{prob:.2f}%"
    except Exception:
        prob_str = "N/A"

    # Format chains
    chains = ",".join(sorted(entry.get("chains", [])))

    # Format residues
    residues = entry.get("residues", {})
    res_list = []
    for chain in sorted(residues):
        sorted_res = sorted(residues[chain])
        res_str = ",".join(map(str, sorted_res))
        res_list.append(f"{chain}:{res_str}")
    residue_str = "; ".join(res_list)

    # Compose final cell content with HTML line breaks
    cell = f"<b>{prob_str}</b><br><i>Chains</i>: {chains}<br><i>Res</i>: {residue_str}"
    pdb_tables[pdb][pocket][model] = cell

# === Generate HTML tables ===
html_blocks = []
for pdb, pocket_data in pdb_tables.items():
    # Filter keys again just in case
    filtered_pocket_data = {k: v for k, v in pocket_data.items() if k and k.strip() != ""}
    df = pd.DataFrame(filtered_pocket_data).T  # pockets as rows
    df = df[sorted(df.columns)]
    df = df.dropna(how='all')

    # Reset index to get rid of any index name artifact
    df = df.reset_index()

    # Rename the first column to 'Pocket'
    df = df.rename(columns={"index": "Pocket"})

    # Convert back to HTML
    html_table = df.fillna("N/A").to_html(index=False, escape=False)  # index=False so no extra index column
    html_blocks.append(f"<h2>{pdb}</h2>\n{html_table}")

# === Combine into single HTML document ===
html_output = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>PDB Model-Pocket Tables</title>
    <style>
        table {{
            border-collapse: collapse;
            margin-bottom: 30px;
            width: 100%;
        }}
        th, td {{
            border: 1px solid #999;
            padding: 8px;
            vertical-align: top;
        }}
        th {{
            background-color: #f2f2f2;
            text-align: center;  /* <-- here */
        }}
        h2 {{
            font-family: Arial, sans-serif;
            margin-top: 40px;
        }}
    </style>
</head>
<body>
{"".join(html_blocks)}
</body>
</html>
"""

# === Write to file ===
output_path = Path("/home/cloetes-lab/amia/test/variant_outputs/variant_pocket_grid_tables.html")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(html_output)

print(f"\n✅ HTML tables saved to: {output_path.resolve()}")



✅ HTML tables saved to: /home/cloetes-lab/amia/test/variant_outputs/variant_pocket_grid_tables.html
